In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os, sys
sys.path.append('/content/drive/MyDrive/CPD_BT')

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import random

import itertools

from bt_cpd import *

import time
import bisect

import pandas as pd

import statsmodels.api as sm
from sklearn import linear_model

from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning
simplefilter("ignore", category=ConvergenceWarning)

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
T = 4
Delta = 800
m = np.array([Delta] * T)
cp_truth = np.cumsum(m)[:T-1]

n = 20
beta = np.zeros((T, n))

t = 0.9
kappa = np.log(t / (1 - t))
delta = 1
beta_ref = get_beta_with_gap(n, delta)
beta_ref *= kappa / (np.max(beta_ref) - np.min(beta_ref))
beta[0] = beta_ref[:]
beta[1] = change_type(beta_ref, 1)
beta[2] = change_type(beta_ref, 2)
beta[3] = change_type(beta_ref, 3)

print(max(beta[0]) - min(beta[0]))

diff = np.zeros(T - 1)
for t in range(1, T):
    diff[t - 1] = np.sum(np.abs(beta[t] - beta[t - 1])**2)**0.5
print(diff)

2.1972245773362196
[5.96433002 5.17173002 5.96433002]


In [5]:
cp_truth

array([ 800, 1600, 2400])

In [6]:
path = '/content/drive/MyDrive/CPD_BT/'
with open(path + 'data_n' + str(n) + '_Delta' + str(Delta) + '_K' + str(T - 1) + '.npy', 'rb') as f:
    X_train_list = np.load(f)
    Y_train_list = np.load(f)
    X_test_list = np.load(f)
    Y_test_list = np.load(f)

In [7]:
X_train_list.shape

(100, 3200, 20)

In [8]:
np.random.seed(0)

grid_n = 90
gamma_list = [20, 40]
lam_list = [0.1]

nt = Delta * T
B = 100

run_time_d = np.zeros(B)
loc_error_d = np.zeros(B)
K_d = np.zeros(B)

cp_best_list = []
param_best_list = []
cp_best_cand_list = []

for b in range(B):
    X_train = X_train_list[b]
    Y_train = Y_train_list[b]
    X_test = X_test_list[b]
    Y_test = Y_test_list[b]

    start_time = time.time()
    dp_fit = dplr_cv_bt(grid_n, lam_list, gamma_list)
    cp_best, param_best, cp_best_cand = dp_fit.fit((Y_train, X_train), (Y_test, X_test))    
    run_time_d[b] = time.time() - start_time
    loc_error_d[b] = cp_distance(cp_best, cp_truth)
    K_d[b] = len(cp_best)

    cp_best_list.append(cp_best)
    param_best_list.append(param_best)
    cp_best_cand_list.append(cp_best_cand)
    print(b)


print('---------- dplr -----------')
print("avg loc error: {0}, avg time: {1}".format(loc_error_d.mean(), run_time_d.mean()))
print("std loc error: {0}, std time: {1}".format(loc_error_d.std(), run_time_d.std()))
print('K < K*: {0}, K = K*: {1}, K > K*: {2}'.format(sum(K_d < T - 1), sum(K_d == T - 1), sum(K_d > T - 1)))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
---------- dplr -----------
avg loc error: 9.0, avg time: 118.53355576276779
std loc error: 9.9498743710662, std time: 2.243416145437494
K < K*: 0, K = K*: 100, K > K*: 0


In [9]:
loc_error_d

array([ 2.,  5., 16.,  8.,  3.,  2., 11., 12.,  4.,  1., 15., 10.,  2.,
        3., 12.,  1., 19.,  7., 20., 19.,  7.,  5.,  6., 35.,  8., 11.,
        6.,  7.,  7.,  6., 11., 13., 31.,  2.,  1.,  3.,  5.,  7.,  4.,
       13., 13.,  1.,  9.,  7.,  4.,  3.,  1.,  9.,  4.,  4.,  4., 10.,
       10., 15.,  4., 22.,  6.,  4.,  3.,  2.,  2.,  2.,  5.,  7.,  9.,
        1.,  6., 11., 26.,  6.,  9., 29.,  1., 13., 24.,  4., 10.,  1.,
        8., 79., 11., 12.,  4.,  9., 28.,  5.,  2.,  0.,  3.,  9.,  3.,
        3.,  8., 10.,  3., 15.,  1.,  9., 11.,  6.])

In [10]:
cp_best_list

[[800, 1600, 2402],
 [799, 1600, 2395],
 [784, 1600, 2397],
 [804, 1608, 2406],
 [803, 1603, 2402],
 [802, 1602, 2401],
 [809, 1595, 2411],
 [800, 1612, 2402],
 [800, 1596, 2400],
 [800, 1601, 2400],
 [801, 1615, 2399],
 [804, 1596, 2390],
 [802, 1598, 2401],
 [800, 1602, 2397],
 [812, 1606, 2410],
 [800, 1601, 2400],
 [797, 1603, 2419],
 [803, 1593, 2397],
 [802, 1580, 2406],
 [800, 1619, 2387],
 [804, 1601, 2393],
 [800, 1600, 2395],
 [803, 1603, 2406],
 [803, 1565, 2396],
 [792, 1597, 2394],
 [791, 1605, 2411],
 [806, 1602, 2400],
 [801, 1600, 2393],
 [807, 1600, 2398],
 [802, 1596, 2394],
 [789, 1603, 2398],
 [799, 1606, 2387],
 [792, 1569, 2393],
 [801, 1601, 2398],
 [800, 1599, 2401],
 [800, 1600, 2397],
 [805, 1599, 2395],
 [805, 1607, 2394],
 [796, 1600, 2402],
 [808, 1600, 2387],
 [803, 1600, 2413],
 [801, 1601, 2399],
 [797, 1591, 2402],
 [799, 1605, 2393],
 [798, 1596, 2396],
 [800, 1600, 2403],
 [800, 1600, 2399],
 [792, 1591, 2403],
 [801, 1599, 2396],
 [804, 1599, 2399],


In [11]:
param_best_list

[(0.1, 40),
 (0.1, 20),
 (0.1, 20),
 (0.1, 40),
 (0.1, 20),
 (0.1, 20),
 (0.1, 40),
 (0.1, 20),
 (0.1, 20),
 (0.1, 40),
 (0.1, 20),
 (0.1, 20),
 (0.1, 40),
 (0.1, 20),
 (0.1, 20),
 (0.1, 20),
 (0.1, 20),
 (0.1, 40),
 (0.1, 40),
 (0.1, 40),
 (0.1, 40),
 (0.1, 20),
 (0.1, 20),
 (0.1, 20),
 (0.1, 20),
 (0.1, 40),
 (0.1, 40),
 (0.1, 20),
 (0.1, 40),
 (0.1, 40),
 (0.1, 40),
 (0.1, 20),
 (0.1, 20),
 (0.1, 40),
 (0.1, 20),
 (0.1, 40),
 (0.1, 40),
 (0.1, 40),
 (0.1, 40),
 (0.1, 40),
 (0.1, 40),
 (0.1, 20),
 (0.1, 20),
 (0.1, 40),
 (0.1, 20),
 (0.1, 40),
 (0.1, 20),
 (0.1, 20),
 (0.1, 20),
 (0.1, 20),
 (0.1, 40),
 (0.1, 20),
 (0.1, 40),
 (0.1, 20),
 (0.1, 20),
 (0.1, 40),
 (0.1, 20),
 (0.1, 20),
 (0.1, 40),
 (0.1, 20),
 (0.1, 20),
 (0.1, 40),
 (0.1, 20),
 (0.1, 40),
 (0.1, 20),
 (0.1, 40),
 (0.1, 40),
 (0.1, 20),
 (0.1, 20),
 (0.1, 20),
 (0.1, 40),
 (0.1, 20),
 (0.1, 40),
 (0.1, 40),
 (0.1, 20),
 (0.1, 40),
 (0.1, 40),
 (0.1, 20),
 (0.1, 20),
 (0.1, 20),
 (0.1, 20),
 (0.1, 40),
 (0.1, 40),
 (0.

In [12]:
import pickle
with open(path + 'dplr_n' + str(n) + '_Delta' + str(Delta) + '_K' + str(T - 1) + '_grid' + str(grid_n) + '.pickle', 'wb') as f:
    pickle.dump([cp_best_list, param_best_list, cp_best_cand_list, loc_error_d, run_time_d, K_d], f)